In [34]:
import os
import matplotlib
import matplotlib.pyplot as plt 
from matplotlib.collections import LineCollection
import shutil
import pandas as pd 
import numpy as np 

In [35]:
class Measurments():
    # инициализация, проверка существования папки, проверка пустоты папки
    def __init__(self, main_folder: str) -> None:
        if not os.path.exists(main_folder):
            raise FileNotFoundError(f'directory \'{main_folder}\' is not exist')
        elif  (len(os.listdir(main_folder)) == 0):
            raise ValueError(f'directory \'{main_folder}\' is empty')
        else:
            self.main_folder = main_folder
            self.list_of_subdirs = os.listdir(main_folder)
            self.main_file_path = os.path.abspath(main_folder)
            
    # поиск всех вложенных директорий в основной папке
    def _find_subdirectories(self) -> None:
        self.subdirs_list = set()
        for i in self.list_of_subdirs:
            if os.path.isdir('\\'.join([self.main_folder, i])):
                self.subdirs_list.add(f'{i}')

    # проверка, что файл типа data
    def _is_data(self, path: str) -> bool:
        if path.split('.')[-1] == 'data':
            return True
        else:
            return False
    # создание пути из названия папок
    def _create_path(self, list: list[str]) -> str:
        return '\\'.join([self.main_file_path] + list)

    # считывание данных об измерениях из файлов во вложенных директориях
    def _create_dict_of_measurments(self, return_dict: bool = False) -> None | dict:
        self._find_subdirectories()
        if len(self.subdirs_list) == 0:
            raise ValueError(f'directory \'{self.main_folder}\' does not contain subdirectories')
        self.dict_of_measurments = {}
        for i in self.subdirs_list:
            subdir_catalogs_list = [file for file in os.listdir(self._create_path([i])) if self._is_data(self._create_path([i, file]))]
            if len(subdir_catalogs_list) == 0:
                continue
            subdir_measurs_type = {}
            for j in subdir_catalogs_list:
                file_path = self._create_path([i, j])
                with open(file_path) as file:
                    measur_type = '_'.join(file.readlines()[1].split()[1:3])
                subdir_measurs_type[j.replace('.data', '')] = measur_type
            self.dict_of_measurments[i] = subdir_measurs_type
        if return_dict == True:
            return self.dict_of_measurments
        
    # удаление папки или папок
    def delete_folders_from_dict(self, folders_name: str | list, return_dict: bool = False) -> dict | None:
        if not hasattr(self, 'dict_of_measurments'):
            self._create_dict_of_measurments()
        folders_from_dict = list(self.dict_of_measurments.keys())
        existed_folders = []
        # проверка типов и существования
        if not isinstance(folders_name, list):
            folders_name = [folders_name]
        for i in folders_name:
            if not isinstance(i, str):
                try:
                    i = str(i)
                except:
                    raise TypeError(f'{i} is not {str} or can\'t be converted to {str}')
            if i in folders_from_dict:
                existed_folders.append(i)
        for i in existed_folders:
            del self.dict_of_measurments[i]
        if return_dict == True:
            return self.dict_of_measurments
        
    # возврат словаря с измерениями
    def get_dict_of_measurments(self)-> dict:
        if hasattr(self, 'dict_of_measurments'):
            return self.dict_of_measurments
        else:
            self._create_dict_of_measurments()
            return self.dict_of_measurments
        
    # абсолютный путь к указанной папке
    def get_abspath(self) -> str:
        return self.main_file_path
    
    # получение словаря только с одного контакта
    def get_contact_dict(self, contact_name: str | int) -> dict:
        if not hasattr(self, 'dict_of_measurments'):
            self._create_dict_of_measurments()
        elif str(contact_name) not in list(self.dict_of_measurments.keys()):
            raise ValueError(f'{contact_name} not exist in {self.main_folder}')
        return self.dict_of_measurments[str(contact_name)]

In [37]:
s = Measurments('hBN_1_2')
s.__dict__

{'main_folder': 'hBN_1_2',
 'list_of_subdirs': ['16',
  '16.header',
  '16_box',
  '16_box.header',
  '17',
  '17.header',
  '26.B',
  '26.B.header',
  'box_test',
  'box_test.header',
  'test',
  'test.header'],
 'main_file_path': 'c:\\Users\\Никита Жидков\\Documents\\GitHub\\data_process\\hBN_1_2'}